# Data Wrangling

Preparacion de datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataset
# df = pd.read_csv('../data/processed/your_processed_data.csv')

# Display the first few rows of the dataset
# df.head()

In [ ]:
# Data Cleaning
# Handle missing values
# df.fillna(method='ffill', inplace=True)

# Data Visualization
# plt.figure(figsize=(10, 6))
# sns.histplot(df['your_column'], bins=30)
# plt.title('Distribution of Your Column')
# plt.show()